In [12]:
import networkx as nx
import qml
import igraph as ig
import graph_tool as gt
import MDAnalysis as mda
import numpy as np

In [13]:
class CompareNX(object):
    def __init__(self, mol2file):
        bonds = mda.topology.MOL2Parser.MOL2Parser(mol2file).parse().bonds.values
        self._g1 = nx.Graph()
        self._g2 = nx.Graph()
        for b1, b2 in bonds:
            self._g1.add_edge(b1, b2)
            self._g2.add_edge(b1, b2)
        self._length = len(self._g1.nodes)
        self._match = nx.algorithms.isomorphism.categorical_node_match('element', 1)
        
    def compare(self, n1, n2):
        for pos in range(self._length):
            self._g1.nodes[pos]['element'] = n1[pos]
            self._g2.nodes[pos]['element'] = n2[pos]
        return nx.algorithms.isomorphism.is_isomorphic(self._g1, self._g2, node_match=self._match)

class CompareQML(object):
    def __init__(self, fn):
        self._c = qml.Compound(fn)
        
    def compare(self, n1, n2):
        r1 = qml.fchl.generate_representation(self._c.coordinates, n1, self._c.natoms)
        r2 = qml.fchl.generate_representation(self._c.coordinates, n2, self._c.natoms)
        return qml.fchl.get_global_kernels(np.array([r1]), np.array([r2]), np.array([2])).flatten()[0] > 0.9999



In [14]:
cmp = CompareNX("benzene.mol2")
cmp = CompareQML("../../test/benzene.xyz")

cmp.compare((6,5,7,7,5,6,1,1,1,1,1,1), (6,7,5,5,7,6,1,1,1,1,1,1)), cmp.compare((6,6,6,6,6,6,1,1,1,1,1,1), (6,6,6,6,6,6,1,1,1,1,1,1))

(False, True)